# Essentia-Tensorflow for Danceability estimation from logits

## Install dependencies and download model weights

In [1]:
!pip install essentia-tensorflow

In [2]:
!wget "https://essentia.upf.edu/models/classification-heads/danceability/danceability-msd-musicnn-1.pb"

--2024-04-15 06:56:44--  https://essentia.upf.edu/models/classification-heads/danceability/danceability-msd-musicnn-1.pb
Resolving essentia.upf.edu (essentia.upf.edu)... 84.89.139.43
Connecting to essentia.upf.edu (essentia.upf.edu)|84.89.139.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 82458 (81K) [application/octet-stream]
Saving to: 'danceability-msd-musicnn-1.pb.2'

danceability-msd-mu 100%[===================>]  80.53K  --.-KB/s    in 0.08s   

2024-04-15 06:56:45 (1013 KB/s) - 'danceability-msd-musicnn-1.pb.2' saved [82458/82458]



In [3]:
!wget "https://essentia.upf.edu/models/feature-extractors/musicnn/msd-musicnn-1.pb"

--2024-04-15 06:56:46--  https://essentia.upf.edu/models/feature-extractors/musicnn/msd-musicnn-1.pb
Resolving essentia.upf.edu (essentia.upf.edu)... 84.89.139.43
Connecting to essentia.upf.edu (essentia.upf.edu)|84.89.139.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3197999 (3.0M) [application/octet-stream]
Saving to: 'msd-musicnn-1.pb.2'

msd-musicnn-1.pb.2  100%[===================>]   3.05M  1.84MB/s    in 1.7s    

2024-04-15 06:56:48 (1.84 MB/s) - 'msd-musicnn-1.pb.2' saved [3197999/3197999]



## Method to extract danceability score estimates from softmax logits

In [4]:
MODEL_PATH = "/kaggle/working/danceability-msd-musicnn-1.pb"
from essentia.standard import MonoLoader, TensorflowPredictMusiCNN, TensorflowPredict2D
from typing import Optional
import numpy as np

[   INFO   ] MusicExtractorSVM: no classifier models were configured by default


In [5]:
import tensorflow as tf 


def get_danceability_score(file_name:str, embedding_model_path:str, danceability_model_path:str) -> Optional[float]:
    # Load audio and get embeddings
    audio = MonoLoader(filename=file_name, sampleRate=16000, resampleQuality=4)()
    embedding_model = TensorflowPredictMusiCNN(graphFilename=embedding_model_path, output="model/dense/BiasAdd")
    embeddings = embedding_model(audio)

    # Load model and get predictions
    model = TensorflowPredict2D(graphFilename=danceability_model_path, output="model/Softmax")
    predictions = model(embeddings)
    mean_danceability_score = np.mean(predictions[:,0])
    return mean_danceability_score

2024-04-15 06:56:49.994536: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-15 06:56:49.994605: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-15 06:56:49.996021: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Running the essentia pipeline on some sample audio files

In [6]:
SAMPLE_FILE_NAME = "/kaggle/input/housex-dataset/consolidatedrawaudio/bass house/Aspyer - Slash.wav"

score = get_danceability_score(file_name=SAMPLE_FILE_NAME, danceability_model_path=MODEL_PATH, embedding_model_path="/kaggle/working/msd-musicnn-1.pb")
print(score)

0.9857686


[   INFO   ] TensorflowPredict: Successfully loaded graph file: `/kaggle/working/msd-musicnn-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `/kaggle/working/danceability-msd-musicnn-1.pb`


In [7]:
SAMPLE_FILE_NAME = "/kaggle/input/housex-dataset/consolidatedrawaudio/melodic house/A7i - Stay.wav"
score = get_danceability_score(file_name=SAMPLE_FILE_NAME, danceability_model_path=MODEL_PATH, embedding_model_path="/kaggle/working/msd-musicnn-1.pb")
print(score)

0.9061129


[   INFO   ] TensorflowPredict: Successfully loaded graph file: `/kaggle/working/msd-musicnn-1.pb`
[   INFO   ] TensorflowPredict: Successfully loaded graph file: `/kaggle/working/danceability-msd-musicnn-1.pb`


In [8]:
SAMPLE_FILE_NAME = "/kaggle/input/housex-dataset/consolidatedrawaudio/progressive house/ARHEX _ HHMR - Eternity.wav"
score = get_danceability_score(file_name=SAMPLE_FILE_NAME, danceability_model_path=MODEL_PATH, embedding_model_path="/kaggle/working/msd-musicnn-1.pb")
print(score)

[   INFO   ] TensorflowPredict: Successfully loaded graph file: `/kaggle/working/msd-musicnn-1.pb`


0.92691034


[   INFO   ] TensorflowPredict: Successfully loaded graph file: `/kaggle/working/danceability-msd-musicnn-1.pb`


In [9]:
from scipy.special import softmax

x = np.array([0.9857686, 0.9061129, 0.92691034])
y = softmax(x)
np.argmax(y)

0